# Report Scraper
This notebook is for collecting the 10K reports from EDGAR, scraping the data, then forming it into a dataframe for later use. 

In [2]:
#import packages
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import qgrid
from urllib.request import urlretrieve
from collections import Counter

##### Provide the URL for EDGAR 

In [3]:
url = "https://www.sec.gov/Archives/edgar/data/812011/0000927356-96-001229.txt"

##### Save the File Locally

In [4]:
file_path = urlretrieve(url, "MTN_10K.txt")[0]

##### Open the Report

In [5]:
TenK = open(file_path, mode="r").readlines()

###  Data Indexing
10K is a 100 page text file. First we need to look for the tables. 
This cell with find the line numbers (start and end) for the table we are scraping. 

1. Define `start` and `end` as the identifiers for finding the start and end lines of the table. 
1. For each line in the report
    1. if the start is in the line, add the line to the start point. 
    1. if the end point is in the line, add it to end points 

In [6]:
start = "(IN THOUSANDS"
end = "The accompanying notes to consolidated financial statements"

start_points = []
end_points = []

for count, line in enumerate(TenK):
    if start in line:
        (start_points.append(count))
    if end in line:
        (end_points.append(count))

print(start_points)
print(end_points)

[1449, 1517, 1593, 1656, 2212]
[1508, 1584, 1647, 1740]


### Create a dictionary of StartPoint $\rightarrow$ EndPoint

In [7]:
all_points = (dict(zip(start_points, end_points)))
all_points

{1449: 1508, 1517: 1584, 1593: 1647, 1656: 1740}

### Partition the Report
1. Go through each start and end line for the tables 
2. Add the lines for each table to `stmt_list`

In [8]:
stmt_list = []

for key, value in all_points.items():
    x = key
    y = value
    stmt_list.append(TenK[x:y])
stmt_list[1]

['               (IN THOUSANDS, EXCEPT SHARE AND PER SHARE AMOUNTS)\n',
 ' \n',
 '<TABLE>\n',
 '<CAPTION>\n',
 '                                          YEAR          YEAR          YEAR\n',
 '                                          ENDED         ENDED         ENDED\n',
 '                                      SEPTEMBER 30, SEPTEMBER 30, SEPTEMBER 30,\n',
 '                                          1994          1995          1996\n',
 '                                      ------------- ------------- -------------\n',
 '<S>                                   <C>           <C>           <C>\n',
 'Net revenues:\n',
 ' Resort.............................   $   124,982   $   126,349   $   140,288\n',
 ' Real estate........................        22,203        16,526        48,655\n',
 '                                       -----------   -----------   -----------\n',
 ' Total net revenues.................       147,185       142,875       188,943\n',
 'Operating expenses:\n',
 ' Resort...

### Newline Character Scraping

1. Strip each line that does not contain data 

In [9]:
new_stmt_list = []

for stmt in stmt_list:
    new_stmt = []
    for line in stmt:
        if "..." in line:
            line = line
            new_stmt.append(line)
        else: 
            line = line.rstrip()
            new_stmt.append(line)
    new_stmt_list.append(new_stmt)
new_stmt_list[1]

['               (IN THOUSANDS, EXCEPT SHARE AND PER SHARE AMOUNTS)',
 '',
 '<TABLE>',
 '<CAPTION>',
 '                                          YEAR          YEAR          YEAR',
 '                                          ENDED         ENDED         ENDED',
 '                                      SEPTEMBER 30, SEPTEMBER 30, SEPTEMBER 30,',
 '                                          1994          1995          1996',
 '                                      ------------- ------------- -------------',
 '<S>                                   <C>           <C>           <C>',
 'Net revenues:',
 ' Resort.............................   $   124,982   $   126,349   $   140,288\n',
 ' Real estate........................        22,203        16,526        48,655\n',
 '                                       -----------   -----------   -----------',
 ' Total net revenues.................       147,185       142,875       188,943\n',
 'Operating expenses:',
 ' Resort............................. 

### Delimiter Scraping
1. Identify the delimiters to split the data with. 
1. Replace the delimiters
1. Add the indices of the delimiters to a list for each report. 
1. Combine all lists of indices for each statement

In [21]:
stmt_table_indices = []
for stmt in stmt_list:
    line_indices = []
    for line in stmt:
        while "<C>" in line:
            line_indices.append(str(len(line.partition("<C>")[0])))
            line = line.replace("<C>", "   ", 1)
    stmt_table_indices.append(line_indices)
print(stmt_table_indices)

[['52', '66'], ['38', '52', '66'], ['26', '37', '47', '58', '65', '76', '86'], ['38', '52', '66']]


## Determine Headers

1. Set regex pattern to identify headers
2. Pull the headers into a list, headers

In [22]:
header_pattern = r"\d\d\d\d\s{5,}\d\d\d\d\s{5,}"
headers = []

for stmt in stmt_list:
    for line in stmt:
        if re.search(header_pattern, line) != None:
            headers.append(line)

## Reformat headers into a list of strings

In [23]:
header_str = str(headers[0].strip())
header_str = re.sub(r"\s+", ",", header_str)
header_str = header_str.split(sep=",")

year1 = header_str[0]
year2 = header_str[1]
year3 = header_str[2]

## Remove unneccesary lines from top of stmts

In [24]:
final_stmt_list = []

for stmt in new_stmt_list:
    for i, line in enumerate(stmt):
        if "<S>" in line:
            x = i
    stmt = stmt[x:]
    final_stmt_list.append(stmt)
    
final_stmt_list[1]

['<S>                                   <C>           <C>           <C>',
 'Net revenues:',
 ' Resort.............................   $   124,982   $   126,349   $   140,288\n',
 ' Real estate........................        22,203        16,526        48,655\n',
 '                                       -----------   -----------   -----------',
 ' Total net revenues.................       147,185       142,875       188,943\n',
 'Operating expenses:',
 ' Resort.............................        78,365        82,305        89,890\n',
 ' Real estate........................        20,341        14,983        40,801\n',
 ' Corporate expense..................         7,160         6,701        12,698\n',
 ' Depreciation and amortization......        17,186        17,968        18,148\n',
 '                                       -----------   -----------   -----------',
 ' Total operating expenses...........       123,052       121,957       161,537\n',
 '                                    

## Place headers

In [25]:

for stmt in final_stmt_list:
    x = stmt[0].count("<C>")
    if x == 2:
        stmt[0] = stmt[0].replace("<C>", year2, 1).replace("<C>", year3).replace("<S>", "LINE ITEMS")
    if x == 3:
        stmt[0] = stmt[0].replace("<C>", year1, 1).replace("<C>", year2, 1).replace("<C>", year3).replace("<S>", "LINE ITEMS")
        
final_stmt_list[0]

['LINE ITEMS                                                 1995           1996',
 '                      ASSETS',
 'Current assets:',
 '  Cash and cash equivalents........................   $ 47,534      $ 12,712\n',
 '  Receivables......................................      5,135         5,741\n',
 '  Inventories......................................      4,221         4,639\n',
 '  Deferred income taxes (Note 8)...................      9,500        17,200\n',
 '  Other current assets.............................      3,716         5,490\n',
 '                                                      --------      --------',
 '    Total current assets...........................     70,106        45,782\n',
 'Property, plant, and equipment, net (Note 6).......    205,151       192,669\n',
 'Real estate held for sale..........................     54,858        88,665\n',
 'Deferred charges and other assets..................      6,106        10,440\n',
 'Intangible assets (Note 6)........

### Scrape the Values
1. For each of the delimiter lists per statement
    1. For each delimiter
        1. Split the line into a list representing a row
        1. add the row to a list 
    1. Make a dataframe. 

In [28]:
stmt_data = []
for i, stmt_indices in enumerate(stmt_table_indices): 
    rows = []
    cur_stmt = final_stmt_list[i] 
    for line in cur_stmt: 
        line = line.strip()
        prev = 0
        row = []
        for indx in stmt_indices:
            indx = int(indx)
            row.append(line[prev:indx])
            prev = indx
        row.append(line[prev:])
        row_clean = list(map(lambda x: x.replace(".", "").replace(",", "").replace("$", ""), row))
        rows.append(row_clean)
    df = pd.DataFrame(rows).set_index([0])
    stmt_data.append(df)
stmt_data[3]

,1,2,3
0,,,
LINE ITEMS,1994,1995,1996
Cash flows from operating activi-,,,
ties:,,,
Net income,28782,3282,4735
Adjustments to reconcile net income,,,
to net cash provided by operating,,,
activities:,,,
Depreciation and amortization,18223,17968,18148
Deferred compensation payments in,,,


## Delete unwanted stmts

In [29]:
for i, stmt in enumerate(stmt_data):
    if len(stmt.columns) > 3:
        x = i
        stmt_data.pop(x)

## Save stmts to an excel doc with each sheet containing a single stmt

In [ ]:
excel_writer = pd.ExcelWriter("output.xlsx")
x = 1
for stmt in stmt_data:
    stmt.to_excel(excel_writer, "Sheet" +str(x))
    x = x + 1
excel_writer.save()